# 天池大赛·印象盐城

## 特征工程

初赛提供2012年1月-2017年10月盐城分车型销量配置数据。
第一阶段需要参赛者预测2017年11月盐城分车型销量数据，第二阶段需要参赛者预测2017年12月盐城分车型销量数据。

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [2]:
train = pd.read_csv('../[new] yancheng_train_20171226.csv', low_memory=False)
test = pd.read_csv('../yancheng_testA_20171225.csv')

In [3]:
test.head()

,predict_date,class_id,predict_quantity
0,201711,103507,NaN
1,201711,124140,NaN
2,201711,125403,NaN
3,201711,136916,NaN
4,201711,169673,NaN


### 一、异常值处理

1.1 发现数据里相同车型在同一月份里有两条记录，这里做简单加和

In [4]:
# labels = ['sale_date','class_id','sale_quantity']
labels = ['sale_date','class_id','brand_id','compartment','type_id','level_id','department_id','TR','gearbox_type','displacement','if_charging',
          'price_level','driven_type_id','fuel_type_id','newenergy_type_id','emission_standards_id','if_MPV_id','if_luxurious_id','power',
          'cylinder_number','engine_torque','car_length','car_width','car_height','total_quality','equipment_quality','rated_passenger',
          'wheelbase','front_track','rear_track']
# trainSale = train[labels]
train = train.groupby(labels).agg('sum').reset_index()
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20099 entries, 0 to 20098
Data columns (total 31 columns):
sale_date                20099 non-null int64
class_id                 20099 non-null int64
brand_id                 20099 non-null int64
compartment              20099 non-null int64
type_id                  20099 non-null int64
level_id                 20099 non-null object
department_id            20099 non-null int64
TR                       20099 non-null object
gearbox_type             20099 non-null object
displacement             20099 non-null float64
if_charging              20099 non-null object
price_level              20099 non-null object
driven_type_id           20099 non-null int64
fuel_type_id             20099 non-null object
newenergy_type_id        20099 non-null int64
emission_standards_id    20099 non-null int64
if_MPV_id                20099 non-null int64
if_luxurious_id          20099 non-null int64
power                    20099 non-null object
cylinder

1.2 三个功率为81/70的条目里共有两款车

    其中一款车有记载的功率为66和81，我们令值为81
    
    另一款车有记载的功率为66，70,81和96，也令值为81

In [5]:
train[train['power']=='81/70']

,sale_date,class_id,brand_id,compartment,type_id,level_id,department_id,TR,gearbox_type,displacement,...,car_length,car_width,car_height,total_quality,equipment_quality,rated_passenger,wheelbase,front_track,rear_track,sale_quantity
12382,201601,175962,750,3,2,1,4,5,MT,1.6,...,4473,1706,1469,1735,1275,5,2603,1460,1500,8
13450,201603,961962,750,3,2,1,4,5,MT,1.6,...,4487,1706,1470,1740,1260,5,2603,1460,1500,5
13478,201604,175962,750,3,2,1,4,5,MT,1.6,...,4473,1706,1469,1735,1275,5,2603,1460,1500,32


In [6]:
train.loc[12382,'power']=81
train.loc[13478,'power']=81
train.loc[13450,'power']=81
train[['power']] = train[['power']].astype('float32')

1.3 发动机扭矩

    有三个功率是'155/140'，就当是155吧
    
    还有一个车型是'-'，给个平均值吧

In [7]:
train[train['engine_torque']=='155/140']

,sale_date,class_id,brand_id,compartment,type_id,level_id,department_id,TR,gearbox_type,displacement,...,car_length,car_width,car_height,total_quality,equipment_quality,rated_passenger,wheelbase,front_track,rear_track,sale_quantity
12382,201601,175962,750,3,2,1,4,5,MT,1.6,...,4473,1706,1469,1735,1275,5,2603,1460,1500,8
13450,201603,961962,750,3,2,1,4,5,MT,1.6,...,4487,1706,1470,1740,1260,5,2603,1460,1500,5
13478,201604,175962,750,3,2,1,4,5,MT,1.6,...,4473,1706,1469,1735,1275,5,2603,1460,1500,32


In [8]:
train.loc[12382,'engine_torque']=155
train.loc[13478,'engine_torque']=155
train.loc[13450,'engine_torque']=155

In [9]:
train[train['engine_torque']=='-']

,sale_date,class_id,brand_id,compartment,type_id,level_id,department_id,TR,gearbox_type,displacement,...,car_length,car_width,car_height,total_quality,equipment_quality,rated_passenger,wheelbase,front_track,rear_track,sale_quantity
16287,201612,527765,236,2,2,5,1,1,AT,0.0,...,3675,1630,1518,1360,1050,4,2360,1405,1400,195
16656,201701,527765,236,2,2,5,1,1,AT,0.0,...,3675,1630,1518,1360,1050,4,2360,1405,1400,6
16657,201701,527765,236,2,2,5,1,1,AT,0.0,...,3675,1630,1518,1360,1058,4,2360,1405,1400,6
17011,201702,527765,236,2,2,5,1,1,AT,0.0,...,3675,1630,1518,1360,1050,4,2360,1405,1400,35
17379,201703,527765,236,2,2,5,1,1,AT,0.0,...,3675,1630,1518,1360,1050,4,2360,1405,1400,59
17747,201704,527765,236,2,2,5,1,1,AT,0.0,...,3675,1630,1518,1360,1050,4,2360,1405,1400,30
17748,201704,527765,236,2,2,5,1,1,AT,0.0,...,3675,1630,1518,1360,1058,4,2360,1405,1400,18
18137,201705,527765,236,2,2,5,1,1,AT,0.0,...,3675,1630,1518,1360,1050,4,2360,1405,1400,6
18138,201705,527765,236,2,2,5,1,1,AT,0.0,...,3675,1630,1518,1360,1058,4,2360,1405,1400,24
18514,201706,527765,236,2,2,5,1,1,AT,0.0,...,3675,1630,1518,1360,1050,4,2360,1405,1400,53


In [10]:
train['engine_torque'][train['engine_torque']=='-']=201.8
train['engine_torque'] = train['engine_torque'].astype('float32')

C:\Users\yudake\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


### 二、特征处理

**训练数据**

In [11]:
classLabels = ['class_id']
trainClass = train[classLabels].drop_duplicates().reset_index(drop=True)
                                                                 
trainClass

,class_id
0,125403
1,136916
2,178529
3,194450
4,198427
5,209945
6,248352
7,281301
8,290854
9,291086


2.1 品牌ID共有42个，进行One-hot编码

In [12]:
train['brand_id'].unique()

array([761, 106,  98, 836,  12, 814, 831, 750, 537, 450, 692, 985, 841,
       638, 872, 953, 304, 783, 637,  75, 923, 497, 813, 290, 807, 864,
       498, 236, 542, 512, 294,  49, 126, 682,  68,  76], dtype=int64)

In [13]:
trainBrand = train[['class_id', 'brand_id']].drop_duplicates()
brand_id_dummies = pd.get_dummies(trainBrand['brand_id'], prefix='brand_id')
trainBrand = pd.concat([trainBrand,brand_id_dummies],axis=1)
trainBrand = trainBrand.drop(['brand_id'],axis=1)
trainBrand = trainBrand.groupby('class_id').agg('sum').reset_index()
trainClass = pd.merge(trainClass, trainBrand, on='class_id')
trainClass

,class_id,brand_id_12,brand_id_49,brand_id_68,brand_id_75,brand_id_76,brand_id_98,brand_id_106,brand_id_126,brand_id_236,...,brand_id_813,brand_id_814,brand_id_831,brand_id_836,brand_id_841,brand_id_864,brand_id_872,brand_id_923,brand_id_953,brand_id_985
0,125403,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,136916,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,178529,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,194450,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,198427,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,209945,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
6,248352,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
7,281301,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,290854,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,291086,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


2.2 厢数有三种取值，进行One-hot编码

In [14]:
train['compartment'].unique()

array([2, 3, 1], dtype=int64)

In [15]:
trainCompartment = train[['class_id', 'compartment']].drop_duplicates()
compartment_dummies = pd.get_dummies(trainCompartment['compartment'], prefix='compartment')
trainCompartment = pd.concat([trainCompartment,compartment_dummies],axis=1)
trainCompartment = trainCompartment.drop(['compartment'],axis=1)
trainCompartment = trainCompartment.groupby('class_id').agg('sum').reset_index()
trainClass = pd.merge(trainClass, trainCompartment, on='class_id')
trainClass

,class_id,brand_id_12,brand_id_49,brand_id_68,brand_id_75,brand_id_76,brand_id_98,brand_id_106,brand_id_126,brand_id_236,...,brand_id_836,brand_id_841,brand_id_864,brand_id_872,brand_id_923,brand_id_953,brand_id_985,compartment_1,compartment_2,compartment_3
0,125403,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,136916,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,178529,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,194450,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
4,198427,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,209945,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,248352,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
7,281301,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,290854,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
9,291086,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


2.3 车型类别有四种

In [16]:
train['type_id'].unique()

array([3, 2, 4, 1], dtype=int64)

In [17]:
trainType = train[['class_id', 'type_id']].drop_duplicates()
type_id_dummies = pd.get_dummies(trainType['type_id'], prefix='type_id')
trainType = pd.concat([trainType,type_id_dummies],axis=1)
trainType = trainType.drop(['type_id'],axis=1)
trainType = trainType.groupby('class_id').agg('sum').reset_index()
trainClass = pd.merge(trainClass, trainType, on='class_id')
trainClass

,class_id,brand_id_12,brand_id_49,brand_id_68,brand_id_75,brand_id_76,brand_id_98,brand_id_106,brand_id_126,brand_id_236,...,brand_id_923,brand_id_953,brand_id_985,compartment_1,compartment_2,compartment_3,type_id_1,type_id_2,type_id_3,type_id_4
0,125403,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,136916,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,1,0,0
2,178529,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1
3,194450,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
4,198427,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
5,209945,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
6,248352,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,1,0,0
7,281301,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
8,290854,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
9,291086,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0


2.4 车型级别有六种

In [18]:
train['level_id'].unique()

array(['2', '-', '1', '4', '3', '5'], dtype=object)

In [19]:
trainLevel = train[['class_id', 'level_id']].drop_duplicates()
level_id_dummies = pd.get_dummies(trainLevel['level_id'], prefix='level_id')
trainLevel = pd.concat([trainLevel,level_id_dummies],axis=1)
trainLevel = trainLevel.drop(['level_id'],axis=1)
trainLevel = trainLevel.groupby('class_id').agg('sum').reset_index()
trainClass = pd.merge(trainClass,trainLevel, on='class_id')
trainClass

,class_id,brand_id_12,brand_id_49,brand_id_68,brand_id_75,brand_id_76,brand_id_98,brand_id_106,brand_id_126,brand_id_236,...,type_id_1,type_id_2,type_id_3,type_id_4,level_id_-,level_id_1,level_id_2,level_id_3,level_id_4,level_id_5
0,125403,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
1,136916,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,1,0,0,0
2,178529,0,0,0,0,0,1,0,0,0,...,0,0,0,1,1,0,0,0,0,0
3,194450,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
4,198427,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
5,209945,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
6,248352,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
7,281301,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
8,290854,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
9,291086,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0


2.5 车型系别有七种

In [20]:
train['department_id'].unique()

array([2, 5, 1, 3, 4, 6, 7], dtype=int64)

In [21]:
trainDepartment = train[['class_id', 'department_id']].drop_duplicates()
department_id_dummies = pd.get_dummies(trainDepartment['department_id'], prefix='department_id')
trainDepartment = pd.concat([trainDepartment,department_id_dummies],axis=1)
trainDepartment = trainDepartment.drop(['department_id'],axis=1)
trainDepartment = trainDepartment.groupby('class_id').agg('sum').reset_index()
trainClass = pd.merge(trainClass, trainDepartment, on='class_id')
trainClass

,class_id,brand_id_12,brand_id_49,brand_id_68,brand_id_75,brand_id_76,brand_id_98,brand_id_106,brand_id_126,brand_id_236,...,level_id_3,level_id_4,level_id_5,department_id_1,department_id_2,department_id_3,department_id_4,department_id_5,department_id_6,department_id_7
0,125403,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,136916,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,178529,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,194450,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,198427,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
5,209945,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
6,248352,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
7,281301,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
8,290854,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9,291086,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


2.6 变速器档位有十种

In [22]:
train['TR'].unique()

array(['6', '4', '5', '8', '7', '0', '5;4', '8;7', '9', '1'], dtype=object)

In [23]:
trainTR = train[['class_id', 'TR']].drop_duplicates()
TR_dummies = pd.get_dummies(trainTR['TR'], prefix='TR')
trainTR = pd.concat([trainTR,TR_dummies],axis=1)
trainTR = trainTR.drop(['TR'],axis=1)
trainTR = trainTR.groupby('class_id').agg('sum').reset_index()
trainClass = pd.merge(trainClass, trainTR, on='class_id')
trainClass

,class_id,brand_id_12,brand_id_49,brand_id_68,brand_id_75,brand_id_76,brand_id_98,brand_id_106,brand_id_126,brand_id_236,...,TR_0,TR_1,TR_4,TR_5,TR_5;4,TR_6,TR_7,TR_8,TR_8;7,TR_9
0,125403,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,136916,0,0,0,0,0,0,1,0,0,...,1,0,1,1,0,1,0,0,0,0
2,178529,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,194450,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,1,1,0,0,0
4,198427,1,0,0,0,0,0,0,0,0,...,0,0,1,1,0,1,0,0,0,0
5,209945,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
6,248352,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,1,0,0,0,0
7,281301,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,0,0
8,290854,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,1,0,0,0,0
9,291086,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1


2.7 变速器形式有七种

In [24]:
train['gearbox_type'].unique()

array(['AT', 'MT', 'DCT', 'CVT', 'MT;AT', 'AT;DCT', 'AMT'], dtype=object)

In [25]:
trainGearbox = train[['class_id', 'gearbox_type']].drop_duplicates()
gearbox_type_dummies = pd.get_dummies(trainGearbox['gearbox_type'], prefix='gearbox_type')
trainGearbox = pd.concat([trainGearbox,gearbox_type_dummies],axis=1)
trainGearbox = trainGearbox.drop(['gearbox_type'],axis=1)
trainGearbox = trainGearbox.groupby('class_id').agg('sum').reset_index()
trainClass = pd.merge(trainClass, trainGearbox, on='class_id')
trainClass

,class_id,brand_id_12,brand_id_49,brand_id_68,brand_id_75,brand_id_76,brand_id_98,brand_id_106,brand_id_126,brand_id_236,...,TR_8,TR_8;7,TR_9,gearbox_type_AMT,gearbox_type_AT,gearbox_type_AT;DCT,gearbox_type_CVT,gearbox_type_DCT,gearbox_type_MT,gearbox_type_MT;AT
0,125403,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,136916,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,1,0,0,0
2,178529,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,194450,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,0
4,198427,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
5,209945,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
6,248352,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
7,281301,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,0
8,290854,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,1,0
9,291086,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,1,0


2.8 排量

    区间缩放

In [26]:
train['displacement'].unique()

array([ 2.4,  3. ,  2. ,  2.5,  1. ,  1.1,  1.2,  1.5,  1.4,  1.6,  1.8,
        2.3,  1.9,  2.7,  2.8,  1.3,  3.6,  3.1,  0. ])

In [27]:
trainDisplacement = train[['class_id', 'displacement']].drop_duplicates().reset_index(drop=True)
trainDisplacement.loc[:,'no_displacement']=0
trainDisplacement['no_displacement'][trainDisplacement['displacement']==0]=1
trainDisplacement['displacement'] = trainDisplacement['displacement'].apply(lambda x: (x-1.0)/(3.6-1.0))
displacement_max = trainDisplacement.groupby(trainDisplacement['class_id'])[['displacement']].agg('max').reset_index()
displacement_max.rename(columns={'displacement':'displacement_max'},inplace=True)
displacement_min = trainDisplacement.groupby(trainDisplacement['class_id'])[['displacement']].agg('min').reset_index()
displacement_min.rename(columns={'displacement':'displacement_min'},inplace=True)
displacement_mean = trainDisplacement.groupby(trainDisplacement['class_id'])[['displacement']].agg('mean').reset_index()
displacement_mean.rename(columns={'displacement':'displacement_mean'},inplace=True)
trainDisplacement = trainDisplacement.drop(['displacement'],axis=1)
trainDisplacement = trainDisplacement.drop_duplicates().reset_index(drop=True)
trainDisplacement = pd.merge(trainDisplacement, displacement_max, on='class_id')
trainDisplacement = pd.merge(trainDisplacement, displacement_min, on='class_id')
trainDisplacement = pd.merge(trainDisplacement, displacement_mean, on='class_id')
trainClass = pd.merge(trainClass, trainDisplacement, on='class_id')
trainClass

# train.loc[:,'no_displacement']=0
# train['no_displacement'][train['displacement']==0]=1
# train['displacement'] = train['displacement'].apply(lambda x: (x-1.0)/(3.6-1.0))
# displacement_dummies = pd.get_dummies(train['displacement'], prefix='displacement')
# train = train.drop(['displacement'],axis=1)
# train = pd.concat([train, displacement_dummies],axis=1)
# train.info()

C:\Users\yudake\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,class_id,brand_id_12,brand_id_49,brand_id_68,brand_id_75,brand_id_76,brand_id_98,brand_id_106,brand_id_126,brand_id_236,...,gearbox_type_AT,gearbox_type_AT;DCT,gearbox_type_CVT,gearbox_type_DCT,gearbox_type_MT,gearbox_type_MT;AT,no_displacement,displacement_max,displacement_min,displacement_mean
0,125403,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0.769231,0.384615,0.567308
1,136916,0,0,0,0,0,0,1,0,0,...,1,0,1,0,0,0,0,0.576923,0.384615,0.500000
2,178529,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0.076923,0.000000,0.038462
3,194450,0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,0,0,0.384615,0.230769,0.307692
4,198427,1,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0.192308,0.115385,0.153846
5,209945,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0.769231,0.384615,0.576923
6,248352,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0.230769,0.153846,0.192308
7,281301,0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,0,0,0.769231,0.153846,0.403846
8,290854,0,0,0,0,0,0,0,0,0,...,1,0,1,0,1,0,0,0.538462,0.192308,0.371795
9,291086,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0.538462,0.192308,0.336538


2.9 是否增压

In [28]:
train['if_charging'].unique()

array(['L', 'T'], dtype=object)

In [29]:
trainCharging = train[['class_id', 'if_charging']].drop_duplicates()
if_charging_dummies = pd.get_dummies(trainCharging['if_charging'], prefix='if_charging')
trainCharging = pd.concat([trainCharging,if_charging_dummies],axis=1)
trainCharging = trainCharging.drop(['if_charging'],axis=1)
trainCharging = trainCharging.groupby('class_id').agg('sum').reset_index()
trainClass = pd.merge(trainClass, trainCharging, on='class_id')
trainClass

,class_id,brand_id_12,brand_id_49,brand_id_68,brand_id_75,brand_id_76,brand_id_98,brand_id_106,brand_id_126,brand_id_236,...,gearbox_type_CVT,gearbox_type_DCT,gearbox_type_MT,gearbox_type_MT;AT,no_displacement,displacement_max,displacement_min,displacement_mean,if_charging_L,if_charging_T
0,125403,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.769231,0.384615,0.567308,1,1
1,136916,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0.576923,0.384615,0.500000,1,0
2,178529,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0.076923,0.000000,0.038462,1,0
3,194450,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0.384615,0.230769,0.307692,1,1
4,198427,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0.192308,0.115385,0.153846,1,1
5,209945,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.769231,0.384615,0.576923,1,1
6,248352,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0.230769,0.153846,0.192308,1,0
7,281301,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0.769231,0.153846,0.403846,1,1
8,290854,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0.538462,0.192308,0.371795,1,1
9,291086,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0.538462,0.192308,0.336538,1,1


2.10 成交段

In [30]:
train['price_level'].unique()

array(['35-50W', '15-20W', '5WL', '5-8W', '8-10W', '25-35W', '20-25W',
       '10-15W', '50-75W'], dtype=object)

In [31]:
trainPrice = train[['class_id', 'price_level']].drop_duplicates()
price_level_dummies = pd.get_dummies(trainPrice['price_level'], prefix='price_level')
trainPrice = pd.concat([trainPrice,price_level_dummies],axis=1)
trainPrice = trainPrice.drop(['price_level'],axis=1)
trainPrice = trainPrice.groupby('class_id').agg('sum').reset_index()
trainClass = pd.merge(trainClass, trainPrice, on='class_id')
trainClass

,class_id,brand_id_12,brand_id_49,brand_id_68,brand_id_75,brand_id_76,brand_id_98,brand_id_106,brand_id_126,brand_id_236,...,if_charging_T,price_level_10-15W,price_level_15-20W,price_level_20-25W,price_level_25-35W,price_level_35-50W,price_level_5-8W,price_level_50-75W,price_level_5WL,price_level_8-10W
0,125403,0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,1,0,0,0,0
1,136916,0,0,0,0,0,0,1,0,0,...,0,0,1,1,1,0,0,0,0,0
2,178529,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,194450,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
4,198427,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
5,209945,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,1,0,0
6,248352,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,1
7,281301,0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,0,0,0,0,0
8,290854,0,0,0,0,0,0,0,0,0,...,1,0,1,1,0,0,0,0,0,0
9,291086,0,0,0,0,0,0,0,0,0,...,1,0,1,1,0,0,0,0,0,0


2.11 驱动形式

In [32]:
train['driven_type_id'].unique()

array([1, 2, 3], dtype=int64)

In [33]:
trainDriven = train[['class_id', 'driven_type_id']].drop_duplicates()
driven_type_id_dummies = pd.get_dummies(trainDriven['driven_type_id'], prefix='driven_type_id')
trainDriven = pd.concat([trainDriven,driven_type_id_dummies],axis=1)
trainDriven = trainDriven.drop(['driven_type_id'],axis=1)
trainDriven = trainDriven.groupby('class_id').agg('sum').reset_index()
trainClass = pd.merge(trainClass, trainDriven, on='class_id')
trainClass

,class_id,brand_id_12,brand_id_49,brand_id_68,brand_id_75,brand_id_76,brand_id_98,brand_id_106,brand_id_126,brand_id_236,...,price_level_20-25W,price_level_25-35W,price_level_35-50W,price_level_5-8W,price_level_50-75W,price_level_5WL,price_level_8-10W,driven_type_id_1,driven_type_id_2,driven_type_id_3
0,125403,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,1,0,0
1,136916,0,0,0,0,0,0,1,0,0,...,1,1,0,0,0,0,0,1,0,0
2,178529,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,0
3,194450,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
4,198427,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
5,209945,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,1,1
6,248352,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,1,0,0
7,281301,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,1,0,0
8,290854,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,1
9,291086,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0


2.12 燃料种类

In [34]:
train['fuel_type_id'].unique()

array(['1', '3', '2', '-', '4'], dtype=object)

In [35]:
trainFuel = train[['class_id', 'fuel_type_id']].drop_duplicates()
fuel_type_id_dummies = pd.get_dummies(trainFuel['fuel_type_id'], prefix='fuel_type_id')
trainFuel = pd.concat([trainFuel,fuel_type_id_dummies],axis=1)
trainFuel = trainFuel.drop(['fuel_type_id'],axis=1)
trainFuel = trainFuel.groupby('class_id').agg('sum').reset_index()
trainClass = pd.merge(trainClass, trainFuel, on='class_id')
trainClass

,class_id,brand_id_12,brand_id_49,brand_id_68,brand_id_75,brand_id_76,brand_id_98,brand_id_106,brand_id_126,brand_id_236,...,price_level_5WL,price_level_8-10W,driven_type_id_1,driven_type_id_2,driven_type_id_3,fuel_type_id_-,fuel_type_id_1,fuel_type_id_2,fuel_type_id_3,fuel_type_id_4
0,125403,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
1,136916,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,1,1,0,0
2,178529,0,0,0,0,0,1,0,0,0,...,1,0,0,1,0,0,1,0,0,0
3,194450,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,1,0,0,0
4,198427,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
5,209945,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,1,0,0,0
6,248352,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,1,0,0,0
7,281301,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
8,290854,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,1,1,0,0
9,291086,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0


2.13 新能源类型

In [36]:
train['newenergy_type_id'].unique()

array([1, 2, 3, 4], dtype=int64)

In [37]:
trainNewenergy = train[['class_id', 'newenergy_type_id']].drop_duplicates()
newenergy_type_id_dummies = pd.get_dummies(trainNewenergy['newenergy_type_id'], prefix='newenergy_type_id')
trainNewenergy = pd.concat([trainNewenergy,newenergy_type_id_dummies],axis=1)
trainNewenergy = trainNewenergy.drop(['newenergy_type_id'],axis=1)
trainNewenergy = trainNewenergy.groupby('class_id').agg('sum').reset_index()
trainClass = pd.merge(trainClass, trainNewenergy, on='class_id')
trainClass

,class_id,brand_id_12,brand_id_49,brand_id_68,brand_id_75,brand_id_76,brand_id_98,brand_id_106,brand_id_126,brand_id_236,...,driven_type_id_3,fuel_type_id_-,fuel_type_id_1,fuel_type_id_2,fuel_type_id_3,fuel_type_id_4,newenergy_type_id_1,newenergy_type_id_2,newenergy_type_id_3,newenergy_type_id_4
0,125403,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
1,136916,0,0,0,0,0,0,1,0,0,...,0,0,1,1,0,0,1,1,0,0
2,178529,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0
3,194450,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,1,0,0,0
4,198427,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
5,209945,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,1,0,0,0
6,248352,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
7,281301,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
8,290854,0,0,0,0,0,0,0,0,0,...,1,0,1,1,0,0,1,1,0,0
9,291086,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0


2.14 排放标准

In [38]:
train['emission_standards_id'].unique()

array([3, 5, 1, 2], dtype=int64)

In [39]:
trainEmission = train[['class_id', 'emission_standards_id']].drop_duplicates()
emission_standards_id_dummies = pd.get_dummies(trainEmission['emission_standards_id'], prefix='emission_standards_id')
trainEmission = pd.concat([trainEmission,emission_standards_id_dummies],axis=1)
trainEmission = trainEmission.drop(['emission_standards_id'],axis=1)
trainEmission = trainEmission.groupby('class_id').agg('sum').reset_index()
trainClass = pd.merge(trainClass, trainEmission, on='class_id')
trainClass

,class_id,brand_id_12,brand_id_49,brand_id_68,brand_id_75,brand_id_76,brand_id_98,brand_id_106,brand_id_126,brand_id_236,...,fuel_type_id_3,fuel_type_id_4,newenergy_type_id_1,newenergy_type_id_2,newenergy_type_id_3,newenergy_type_id_4,emission_standards_id_1,emission_standards_id_2,emission_standards_id_3,emission_standards_id_5
0,125403,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
1,136916,0,0,0,0,0,0,1,0,0,...,0,0,1,1,0,0,1,0,1,0
2,178529,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,1,0,1,0
3,194450,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
4,198427,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
5,209945,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
6,248352,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
7,281301,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
8,290854,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,1,0,1,0
9,291086,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0


2.15 是否微客MPV

In [40]:
train['if_MPV_id'].unique()

array([2, 1], dtype=int64)

In [41]:
trainMPV = train[['class_id', 'if_MPV_id']].drop_duplicates()
if_MPV_id_dummies = pd.get_dummies(trainMPV['if_MPV_id'], prefix='if_MPV_id')
trainMPV = pd.concat([trainMPV,if_MPV_id_dummies],axis=1)
trainMPV = trainMPV.drop(['if_MPV_id'],axis=1)
trainMPV = trainMPV.groupby('class_id').agg('sum').reset_index()
trainClass = pd.merge(trainClass, trainMPV, on='class_id')
trainClass

,class_id,brand_id_12,brand_id_49,brand_id_68,brand_id_75,brand_id_76,brand_id_98,brand_id_106,brand_id_126,brand_id_236,...,newenergy_type_id_1,newenergy_type_id_2,newenergy_type_id_3,newenergy_type_id_4,emission_standards_id_1,emission_standards_id_2,emission_standards_id_3,emission_standards_id_5,if_MPV_id_1,if_MPV_id_2
0,125403,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,1,0,0,1
1,136916,0,0,0,0,0,0,1,0,0,...,1,1,0,0,1,0,1,0,0,1
2,178529,0,0,0,0,0,1,0,0,0,...,1,0,0,0,1,0,1,0,0,1
3,194450,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,1,0,0,1
4,198427,1,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,1,0,0,1
5,209945,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,1,0,0,1
6,248352,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,1,0,0,1
7,281301,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,1,0,0,1
8,290854,0,0,0,0,0,0,0,0,0,...,1,1,0,0,1,0,1,0,0,1
9,291086,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,1,0,0,1


2.16 是否豪华

In [42]:
train['if_luxurious_id'].unique()

array([1, 2], dtype=int64)

In [43]:
trainLuxurious = train[['class_id', 'if_luxurious_id']].drop_duplicates()
if_luxurious_id_dummies = pd.get_dummies(trainLuxurious['if_luxurious_id'], prefix='if_luxurious_id')
trainLuxurious = pd.concat([trainLuxurious,if_luxurious_id_dummies],axis=1)
trainLuxurious = trainLuxurious.drop(['if_luxurious_id'],axis=1)
trainLuxurious = trainLuxurious.groupby('class_id').agg('sum').reset_index()
trainClass = pd.merge(trainClass, trainLuxurious, on='class_id')
trainClass

,class_id,brand_id_12,brand_id_49,brand_id_68,brand_id_75,brand_id_76,brand_id_98,brand_id_106,brand_id_126,brand_id_236,...,newenergy_type_id_3,newenergy_type_id_4,emission_standards_id_1,emission_standards_id_2,emission_standards_id_3,emission_standards_id_5,if_MPV_id_1,if_MPV_id_2,if_luxurious_id_1,if_luxurious_id_2
0,125403,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,1,0
1,136916,0,0,0,0,0,0,1,0,0,...,0,0,1,0,1,0,0,1,1,0
2,178529,0,0,0,0,0,1,0,0,0,...,0,0,1,0,1,0,0,1,1,0
3,194450,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,1,0
4,198427,1,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,1,0
5,209945,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,0,1
6,248352,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,1,0
7,281301,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,1,0
8,290854,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,1,0
9,291086,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,1,0


2.17 功率

In [44]:
train['power'].unique()

array([ 123.        ,  190.        ,  108.        ,  109.        ,
        135.        ,   45.        ,   50.        ,   60.        ,
         63.        ,  104.        ,   78.        ,   80.        ,
        144.        ,  167.        ,  211.        ,  250.        ,
         78.69999695,   90.40000153,   96.        ,  118.        ,
        147.        ,  110.        ,  125.        ,  166.        ,
        137.        ,  103.        ,   90.        ,   74.        ,
         86.        ,   89.        ,  105.        ,  126.        ,
         98.        ,   77.        ,   88.        ,  112.        ,
        160.        ,  115.        ,  132.        ,  155.        ,
        150.        ,  180.        ,  171.        ,   83.        ,
         82.        ,   81.58000183,   93.        ,  140.        ,
        121.        ,  130.        ,  162.        ,  213.        ,
         73.        ,  114.        ,  120.        ,  128.        ,
        106.        ,   70.        ,   68.        ,  184.     

In [45]:
trainPower = train[['class_id', 'power']].drop_duplicates().reset_index(drop=True)
trainPower['power'] = MinMaxScaler().fit_transform(trainPower['power'])
power_max = trainPower.groupby(trainPower['class_id'])[['power']].agg('max').reset_index()
power_max.rename(columns={'power':'power_max'},inplace=True)
power_min = trainPower.groupby(trainPower['class_id'])[['power']].agg('min').reset_index()
power_min.rename(columns={'power':'power_min'},inplace=True)
power_mean = trainPower.groupby(trainPower['class_id'])[['power']].agg('mean').reset_index()
power_mean.rename(columns={'power':'power_mean'},inplace=True)
trainPower = trainPower.drop(['power'],axis=1)
trainPower = trainPower.drop_duplicates().reset_index(drop=True)
trainPower = pd.merge(trainPower, power_max, on='class_id')
trainPower = pd.merge(trainPower, power_min, on='class_id')
trainPower = pd.merge(trainPower, power_mean, on='class_id')
trainClass = pd.merge(trainClass, trainPower, on='class_id')
trainClass

C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


,class_id,brand_id_12,brand_id_49,brand_id_68,brand_id_75,brand_id_76,brand_id_98,brand_id_106,brand_id_126,brand_id_236,...,emission_standards_id_2,emission_standards_id_3,emission_standards_id_5,if_MPV_id_1,if_MPV_id_2,if_luxurious_id_1,if_luxurious_id_2,power_max,power_min,power_mean
0,125403,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,0,0.731818,4.090909e-01,0.575758
1,136916,0,0,0,0,0,0,1,0,0,...,0,1,0,0,1,1,0,0.477273,3.545454e-01,0.396212
2,178529,0,0,0,0,0,1,0,0,0,...,0,1,0,0,1,1,0,0.150000,6.818181e-02,0.121061
3,194450,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,0,0.454545,3.363636e-01,0.401515
4,198427,1,0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,0,0.309091,2.181818e-01,0.251515
5,209945,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,1,1.000000,5.181818e-01,0.696970
6,248352,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,0,0.274545,1.968182e-01,0.241591
7,281301,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,0,0.700000,3.000000e-01,0.479870
8,290854,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,0,0.509091,3.500000e-01,0.431818
9,291086,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,0,0.713636,3.545454e-01,0.496970


2.17 缸数

In [46]:
train['cylinder_number'].unique()

array([4, 6, 3, 0], dtype=int64)

In [47]:
trainCylinder = train[['class_id', 'cylinder_number']].drop_duplicates()
cylinder_number_dummies = pd.get_dummies(trainCylinder['cylinder_number'], prefix='cylinder_number')
trainCylinder = pd.concat([trainCylinder,cylinder_number_dummies],axis=1)
trainCylinder = trainCylinder.drop(['cylinder_number'],axis=1)
trainCylinder = trainCylinder.groupby('class_id').agg('sum').reset_index()
trainClass = pd.merge(trainClass, trainCylinder, on='class_id')
trainClass

,class_id,brand_id_12,brand_id_49,brand_id_68,brand_id_75,brand_id_76,brand_id_98,brand_id_106,brand_id_126,brand_id_236,...,if_MPV_id_2,if_luxurious_id_1,if_luxurious_id_2,power_max,power_min,power_mean,cylinder_number_0,cylinder_number_3,cylinder_number_4,cylinder_number_6
0,125403,0,0,0,0,0,0,0,0,0,...,1,1,0,0.731818,4.090909e-01,0.575758,0,0,1,1
1,136916,0,0,0,0,0,0,1,0,0,...,1,1,0,0.477273,3.545454e-01,0.396212,0,0,1,0
2,178529,0,0,0,0,0,1,0,0,0,...,1,1,0,0.150000,6.818181e-02,0.121061,0,0,1,0
3,194450,0,0,0,0,0,0,0,0,0,...,1,1,0,0.454545,3.363636e-01,0.401515,0,0,1,0
4,198427,1,0,0,0,0,0,0,0,0,...,1,1,0,0.309091,2.181818e-01,0.251515,0,0,1,0
5,209945,0,0,0,0,0,0,0,0,0,...,1,0,1,1.000000,5.181818e-01,0.696970,0,0,1,1
6,248352,0,0,0,0,0,0,0,0,0,...,1,1,0,0.274545,1.968182e-01,0.241591,0,0,1,0
7,281301,0,0,0,0,0,0,0,0,0,...,1,1,0,0.700000,3.000000e-01,0.479870,0,0,1,1
8,290854,0,0,0,0,0,0,0,0,0,...,1,1,0,0.509091,3.500000e-01,0.431818,0,0,1,0
9,291086,0,0,0,0,0,0,0,0,0,...,1,1,0,0.713636,3.545454e-01,0.496970,0,0,1,0


2.18 发动机扭矩

In [48]:
trainEngine = train[['class_id', 'engine_torque']].drop_duplicates().reset_index(drop=True)
trainEngine['engine_torque'] = MinMaxScaler().fit_transform(trainEngine['engine_torque'])
engine_torque_max = trainEngine.groupby(trainEngine['class_id'])[['engine_torque']].agg('max').reset_index()
engine_torque_max.rename(columns={'engine_torque':'engine_torque_max'},inplace=True)
engine_torque_min = trainEngine.groupby(trainEngine['class_id'])[['engine_torque']].agg('min').reset_index()
engine_torque_min.rename(columns={'engine_torque':'engine_torque_min'},inplace=True)
engine_torque_mean = trainEngine.groupby(trainEngine['class_id'])[['engine_torque']].agg('mean').reset_index()
engine_torque_mean.rename(columns={'engine_torque':'engine_torque_mean'},inplace=True)
trainEngine = trainEngine.drop(['engine_torque'],axis=1)
trainEngine = trainEngine.drop_duplicates().reset_index(drop=True)
trainEngine = pd.merge(trainEngine, engine_torque_max, on='class_id')
trainEngine = pd.merge(trainEngine, engine_torque_min, on='class_id')
trainEngine = pd.merge(trainEngine, engine_torque_mean, on='class_id')
trainClass = pd.merge(trainClass, trainEngine, on='class_id')
trainClass

C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


,class_id,brand_id_12,brand_id_49,brand_id_68,brand_id_75,brand_id_76,brand_id_98,brand_id_106,brand_id_126,brand_id_236,...,power_max,power_min,power_mean,cylinder_number_0,cylinder_number_3,cylinder_number_4,cylinder_number_6,engine_torque_max,engine_torque_min,engine_torque_mean
0,125403,0,0,0,0,0,0,0,0,0,...,0.731818,4.090909e-01,0.575758,0,0,1,1,0.746479,3.943661e-01,0.525634
1,136916,0,0,0,0,0,0,1,0,0,...,0.477273,3.545454e-01,0.396212,0,0,1,0,0.422535,2.873239e-01,0.346479
2,178529,0,0,0,0,0,1,0,0,0,...,0.150000,6.818181e-02,0.121061,0,0,1,0,0.087324,-1.490116e-08,0.050235
3,194450,0,0,0,0,0,0,0,0,0,...,0.454545,3.363636e-01,0.401515,0,0,1,0,0.507042,2.788732e-01,0.372770
4,198427,1,0,0,0,0,0,0,0,0,...,0.309091,2.181818e-01,0.251515,0,0,1,0,0.281690,1.549295e-01,0.199061
5,209945,0,0,0,0,0,0,0,0,0,...,1.000000,5.181818e-01,0.696970,0,0,1,1,0.887324,4.084507e-01,0.610329
6,248352,0,0,0,0,0,0,0,0,0,...,0.274545,1.968182e-01,0.241591,0,0,1,0,0.197183,1.335211e-01,0.164155
7,281301,0,0,0,0,0,0,0,0,0,...,0.700000,3.000000e-01,0.479870,0,0,1,1,0.746479,3.802817e-01,0.563380
8,290854,0,0,0,0,0,0,0,0,0,...,0.509091,3.500000e-01,0.431818,0,0,1,0,0.445070,2.535211e-01,0.352113
9,291086,0,0,0,0,0,0,0,0,0,...,0.713636,3.545454e-01,0.496970,0,0,1,0,0.746479,2.957746e-01,0.453521


2.19 车长

In [49]:
trainLength = train[['class_id', 'car_length']].drop_duplicates().reset_index(drop=True)
trainLength['car_length'] = MinMaxScaler().fit_transform(trainLength['car_length'])
car_length_max = trainLength.groupby(trainLength['class_id'])[['car_length']].agg('max').reset_index()
car_length_max.rename(columns={'car_length':'car_length_max'},inplace=True)
car_length_min = trainLength.groupby(trainLength['class_id'])[['car_length']].agg('min').reset_index()
car_length_min.rename(columns={'car_length':'car_length_min'},inplace=True)
car_length_mean = trainLength.groupby(trainLength['class_id'])[['car_length']].agg('mean').reset_index()
car_length_mean.rename(columns={'car_length':'car_length_mean'},inplace=True)
trainLength = trainLength.drop(['car_length'],axis=1)
trainLength = trainLength.drop_duplicates().reset_index(drop=True)
trainLength = pd.merge(trainLength, car_length_max, on='class_id')
trainLength = pd.merge(trainLength, car_length_min, on='class_id')
trainLength = pd.merge(trainLength, car_length_mean, on='class_id')
trainClass = pd.merge(trainClass, trainLength, on='class_id')
trainClass

C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


,class_id,brand_id_12,brand_id_49,brand_id_68,brand_id_75,brand_id_76,brand_id_98,brand_id_106,brand_id_126,brand_id_236,...,cylinder_number_0,cylinder_number_3,cylinder_number_4,cylinder_number_6,engine_torque_max,engine_torque_min,engine_torque_mean,car_length_max,car_length_min,car_length_mean
0,125403,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0.746479,3.943661e-01,0.525634,1.000000,0.960402,0.980201
1,136916,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0.422535,2.873239e-01,0.346479,0.738529,0.722816,0.730673
2,178529,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0.087324,-1.490116e-08,0.050235,0.370836,0.034569,0.166741
3,194450,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0.507042,2.788732e-01,0.372770,0.502828,0.408548,0.455688
4,198427,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0.281690,1.549295e-01,0.199061,0.582652,0.548083,0.568615
5,209945,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0.887324,4.084507e-01,0.610329,0.887492,0.857322,0.870731
6,248352,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0.197183,1.335211e-01,0.164155,0.455688,0.279698,0.390740
7,281301,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0.746479,3.802817e-01,0.563380,0.752357,0.751100,0.751728
8,290854,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0.445070,2.535211e-01,0.352113,0.571967,0.537398,0.556254
9,291086,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0.746479,2.957746e-01,0.453521,0.771213,0.725959,0.739943


2.20 车宽

In [50]:
trainWidth = train[['class_id', 'car_width']].drop_duplicates().reset_index(drop=True)
trainWidth['car_width'] = MinMaxScaler().fit_transform(trainWidth['car_width'])
car_width_max = trainWidth.groupby(trainWidth['class_id'])[['car_width']].agg('max').reset_index()
car_width_max.rename(columns={'car_width':'car_width_max'},inplace=True)
car_width_min = trainWidth.groupby(trainWidth['class_id'])[['car_width']].agg('min').reset_index()
car_width_min.rename(columns={'car_width':'car_width_min'},inplace=True)
car_width_mean = trainWidth.groupby(trainWidth['class_id'])[['car_width']].agg('mean').reset_index()
car_width_mean.rename(columns={'car_width':'car_width_mean'},inplace=True)
trainWidth = trainWidth.drop(['car_width'],axis=1)
trainWidth = trainWidth.drop_duplicates().reset_index(drop=True)
trainWidth = pd.merge(trainWidth, car_width_max, on='class_id')
trainWidth = pd.merge(trainWidth, car_width_min, on='class_id')
trainWidth = pd.merge(trainWidth, car_width_mean, on='class_id')
trainClass = pd.merge(trainClass, trainWidth, on='class_id')
trainClass

C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


,class_id,brand_id_12,brand_id_49,brand_id_68,brand_id_75,brand_id_76,brand_id_98,brand_id_106,brand_id_126,brand_id_236,...,cylinder_number_6,engine_torque_max,engine_torque_min,engine_torque_mean,car_length_max,car_length_min,car_length_mean,car_width_max,car_width_min,car_width_mean
0,125403,0,0,0,0,0,0,0,0,0,...,1,0.746479,3.943661e-01,0.525634,1.000000,0.960402,0.980201,0.851852,0.780093,0.815972
1,136916,0,0,0,0,0,0,1,0,0,...,0,0.422535,2.873239e-01,0.346479,0.738529,0.722816,0.730673,0.729167,0.717593,0.723380
2,178529,0,0,0,0,0,1,0,0,0,...,0,0.087324,-1.490116e-08,0.050235,0.370836,0.034569,0.166741,0.243056,0.000000,0.150463
3,194450,0,0,0,0,0,0,0,0,0,...,0,0.507042,2.788732e-01,0.372770,0.502828,0.408548,0.455688,0.787037,0.659722,0.723380
4,198427,1,0,0,0,0,0,0,0,0,...,0,0.281690,1.549295e-01,0.199061,0.582652,0.548083,0.568615,0.518519,0.497685,0.508102
5,209945,0,0,0,0,0,0,0,0,0,...,1,0.887324,4.084507e-01,0.610329,0.887492,0.857322,0.870731,0.828704,0.810185,0.819444
6,248352,0,0,0,0,0,0,0,0,0,...,0,0.197183,1.335211e-01,0.164155,0.455688,0.279698,0.390740,0.532407,0.439815,0.486111
7,281301,0,0,0,0,0,0,0,0,0,...,1,0.746479,3.802817e-01,0.563380,0.752357,0.751100,0.751728,0.750000,0.750000,0.750000
8,290854,0,0,0,0,0,0,0,0,0,...,0,0.445070,2.535211e-01,0.352113,0.571967,0.537398,0.556254,0.798611,0.717593,0.758102
9,291086,0,0,0,0,0,0,0,0,0,...,0,0.746479,2.957746e-01,0.453521,0.771213,0.725959,0.739943,0.817130,0.800926,0.809028


2.21 车高

In [51]:
trainHeight = train[['class_id', 'car_height']].drop_duplicates().reset_index(drop=True)
trainHeight['car_height'] = MinMaxScaler().fit_transform(trainHeight['car_height'])
car_height_max = trainHeight.groupby(trainHeight['class_id'])[['car_height']].agg('max').reset_index()
car_height_max.rename(columns={'car_height':'car_height_max'},inplace=True)
car_height_min = trainHeight.groupby(trainHeight['class_id'])[['car_height']].agg('min').reset_index()
car_height_min.rename(columns={'car_height':'car_height_min'},inplace=True)
car_height_mean = trainHeight.groupby(trainHeight['class_id'])[['car_height']].agg('mean').reset_index()
car_height_mean.rename(columns={'car_height':'car_height_mean'},inplace=True)
trainHeight = trainHeight.drop(['car_height'],axis=1)
trainHeight = trainHeight.drop_duplicates().reset_index(drop=True)
trainHeight = pd.merge(trainHeight, car_height_max, on='class_id')
trainHeight = pd.merge(trainHeight, car_height_min, on='class_id')
trainHeight = pd.merge(trainHeight, car_height_mean, on='class_id')
trainClass = pd.merge(trainClass, trainHeight, on='class_id')
trainClass

C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


,class_id,brand_id_12,brand_id_49,brand_id_68,brand_id_75,brand_id_76,brand_id_98,brand_id_106,brand_id_126,brand_id_236,...,engine_torque_mean,car_length_max,car_length_min,car_length_mean,car_width_max,car_width_min,car_width_mean,car_height_max,car_height_min,car_height_mean
0,125403,0,0,0,0,0,0,0,0,0,...,0.525634,1.000000,0.960402,0.980201,0.851852,0.780093,0.815972,0.726606,0.614679,0.682875
1,136916,0,0,0,0,0,0,1,0,0,...,0.346479,0.738529,0.722816,0.730673,0.729167,0.717593,0.723380,0.128440,0.108257,0.118654
2,178529,0,0,0,0,0,1,0,0,0,...,0.050235,0.370836,0.034569,0.166741,0.243056,0.000000,0.150463,0.816514,0.743119,0.773700
3,194450,0,0,0,0,0,0,0,0,0,...,0.372770,0.502828,0.408548,0.455688,0.787037,0.659722,0.723380,0.486239,0.440367,0.463303
4,198427,1,0,0,0,0,0,0,0,0,...,0.199061,0.582652,0.548083,0.568615,0.518519,0.497685,0.508102,0.128440,0.100917,0.114679
5,209945,0,0,0,0,0,0,0,0,0,...,0.610329,0.887492,0.857322,0.870731,0.828704,0.810185,0.819444,0.155963,0.102752,0.134862
6,248352,0,0,0,0,0,0,0,0,0,...,0.164155,0.455688,0.279698,0.390740,0.532407,0.439815,0.486111,0.082569,0.082569,0.082569
7,281301,0,0,0,0,0,0,0,0,0,...,0.563380,0.752357,0.751100,0.751728,0.750000,0.750000,0.750000,0.126606,0.104587,0.115596
8,290854,0,0,0,0,0,0,0,0,0,...,0.352113,0.571967,0.537398,0.556254,0.798611,0.717593,0.758102,0.502752,0.484404,0.492202
9,291086,0,0,0,0,0,0,0,0,0,...,0.453521,0.771213,0.725959,0.739943,0.817130,0.800926,0.809028,0.144954,0.075229,0.107798


2.22 质量

In [52]:
trainTQuality = train[['class_id', 'total_quality']].drop_duplicates().reset_index(drop=True)
trainTQuality['total_quality'] = MinMaxScaler().fit_transform(trainTQuality['total_quality'])
total_quality_max = trainTQuality.groupby(trainTQuality['class_id'])[['total_quality']].agg('max').reset_index()
total_quality_max.rename(columns={'total_quality':'total_quality_max'},inplace=True)
total_quality_min = trainTQuality.groupby(trainTQuality['class_id'])[['total_quality']].agg('min').reset_index()
total_quality_min.rename(columns={'total_quality':'total_quality_min'},inplace=True)
total_quality_mean = trainTQuality.groupby(trainTQuality['class_id'])[['total_quality']].agg('mean').reset_index()
total_quality_mean.rename(columns={'total_quality':'total_quality_mean'},inplace=True)
trainTQuality = trainTQuality.drop(['total_quality'],axis=1)
trainTQuality = trainTQuality.drop_duplicates().reset_index(drop=True)
trainTQuality = pd.merge(trainTQuality, total_quality_max, on='class_id')
trainTQuality = pd.merge(trainTQuality, total_quality_min, on='class_id')
trainTQuality = pd.merge(trainTQuality, total_quality_mean, on='class_id')
trainClass = pd.merge(trainClass, trainTQuality, on='class_id')
trainClass

C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


,class_id,brand_id_12,brand_id_49,brand_id_68,brand_id_75,brand_id_76,brand_id_98,brand_id_106,brand_id_126,brand_id_236,...,car_length_mean,car_width_max,car_width_min,car_width_mean,car_height_max,car_height_min,car_height_mean,total_quality_max,total_quality_min,total_quality_mean
0,125403,0,0,0,0,0,0,0,0,0,...,0.980201,0.851852,0.780093,0.815972,0.726606,0.614679,0.682875,1.000000,0.880342,0.945055
1,136916,0,0,0,0,0,0,1,0,0,...,0.730673,0.729167,0.717593,0.723380,0.128440,0.108257,0.118654,0.666667,0.555556,0.621083
2,178529,0,0,0,0,0,1,0,0,0,...,0.166741,0.243056,0.000000,0.150463,0.816514,0.743119,0.773700,0.384615,0.170940,0.269231
3,194450,0,0,0,0,0,0,0,0,0,...,0.455688,0.787037,0.659722,0.723380,0.486239,0.440367,0.463303,0.683761,0.461538,0.559829
4,198427,1,0,0,0,0,0,0,0,0,...,0.568615,0.518519,0.497685,0.508102,0.128440,0.100917,0.114679,0.230769,0.222222,0.227066
5,209945,0,0,0,0,0,0,0,0,0,...,0.870731,0.828704,0.810185,0.819444,0.155963,0.102752,0.134862,0.858974,0.730769,0.791764
6,248352,0,0,0,0,0,0,0,0,0,...,0.390740,0.532407,0.439815,0.486111,0.082569,0.082569,0.082569,0.179487,0.094017,0.136752
7,281301,0,0,0,0,0,0,0,0,0,...,0.751728,0.750000,0.750000,0.750000,0.126606,0.104587,0.115596,0.705128,0.534188,0.616667
8,290854,0,0,0,0,0,0,0,0,0,...,0.556254,0.798611,0.717593,0.758102,0.502752,0.484404,0.492202,0.649573,0.564103,0.611111
9,291086,0,0,0,0,0,0,0,0,0,...,0.739943,0.817130,0.800926,0.809028,0.144954,0.075229,0.107798,0.636752,0.470085,0.570750


2.23 装备质量

In [53]:
trainEQuality = train[['class_id', 'equipment_quality']].drop_duplicates().reset_index(drop=True)
trainEQuality['equipment_quality'] = MinMaxScaler().fit_transform(trainEQuality['equipment_quality'])
equipment_quality_max = trainEQuality.groupby(trainEQuality['class_id'])[['equipment_quality']].agg('max').reset_index()
equipment_quality_max.rename(columns={'equipment_quality':'equipment_quality_max'},inplace=True)
equipment_quality_min = trainEQuality.groupby(trainEQuality['class_id'])[['equipment_quality']].agg('min').reset_index()
equipment_quality_min.rename(columns={'equipment_quality':'equipment_quality_min'},inplace=True)
equipment_quality_mean = trainEQuality.groupby(trainEQuality['class_id'])[['equipment_quality']].agg('mean').reset_index()
equipment_quality_mean.rename(columns={'equipment_quality':'equipment_quality_mean'},inplace=True)
trainEQuality = trainEQuality.drop(['equipment_quality'],axis=1)
trainEQuality = trainEQuality.drop_duplicates().reset_index(drop=True)
trainEQuality = pd.merge(trainEQuality, equipment_quality_max, on='class_id')
trainEQuality = pd.merge(trainEQuality, equipment_quality_min, on='class_id')
trainEQuality = pd.merge(trainEQuality, equipment_quality_mean, on='class_id')
trainClass = pd.merge(trainClass, trainEQuality, on='class_id')
trainClass

C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


,class_id,brand_id_12,brand_id_49,brand_id_68,brand_id_75,brand_id_76,brand_id_98,brand_id_106,brand_id_126,brand_id_236,...,car_width_mean,car_height_max,car_height_min,car_height_mean,total_quality_max,total_quality_min,total_quality_mean,equipment_quality_max,equipment_quality_min,equipment_quality_mean
0,125403,0,0,0,0,0,0,0,0,0,...,0.815972,0.726606,0.614679,0.682875,1.000000,0.880342,0.945055,0.976852,0.847222,0.910450
1,136916,0,0,0,0,0,0,1,0,0,...,0.723380,0.128440,0.108257,0.118654,0.666667,0.555556,0.621083,0.689815,0.467593,0.551257
2,178529,0,0,0,0,0,1,0,0,0,...,0.150463,0.816514,0.743119,0.773700,0.384615,0.170940,0.269231,0.134259,0.000000,0.053241
3,194450,0,0,0,0,0,0,0,0,0,...,0.723380,0.486239,0.440367,0.463303,0.683761,0.461538,0.559829,0.650000,0.519444,0.579894
4,198427,1,0,0,0,0,0,0,0,0,...,0.508102,0.128440,0.100917,0.114679,0.230769,0.222222,0.227066,0.293519,0.254630,0.270679
5,209945,0,0,0,0,0,0,0,0,0,...,0.819444,0.155963,0.102752,0.134862,0.858974,0.730769,0.791764,0.879630,0.680556,0.789062
6,248352,0,0,0,0,0,0,0,0,0,...,0.486111,0.082569,0.082569,0.082569,0.179487,0.094017,0.136752,0.203704,0.105556,0.145885
7,281301,0,0,0,0,0,0,0,0,0,...,0.750000,0.126606,0.104587,0.115596,0.705128,0.534188,0.616667,0.675926,0.472222,0.563552
8,290854,0,0,0,0,0,0,0,0,0,...,0.758102,0.502752,0.484404,0.492202,0.649573,0.564103,0.611111,0.679630,0.509259,0.600195
9,291086,0,0,0,0,0,0,0,0,0,...,0.809028,0.144954,0.075229,0.107798,0.636752,0.470085,0.570750,0.680556,0.467593,0.600926


2.24 轴距

In [54]:
trainWheel = train[['class_id', 'wheelbase']].drop_duplicates().reset_index(drop=True)
trainWheel['wheelbase'] = MinMaxScaler().fit_transform(trainWheel['wheelbase'])
wheelbase_max = trainWheel.groupby(trainWheel['class_id'])[['wheelbase']].agg('max').reset_index()
wheelbase_max.rename(columns={'wheelbase':'wheelbase_max'},inplace=True)
wheelbase_min = trainWheel.groupby(trainWheel['class_id'])[['wheelbase']].agg('min').reset_index()
wheelbase_min.rename(columns={'wheelbase':'wheelbase_min'},inplace=True)
wheelbase_mean = trainWheel.groupby(trainWheel['class_id'])[['wheelbase']].agg('mean').reset_index()
wheelbase_mean.rename(columns={'wheelbase':'wheelbase_mean'},inplace=True)
trainWheel = trainWheel.drop(['wheelbase'],axis=1)
trainWheel = trainWheel.drop_duplicates().reset_index(drop=True)
trainWheel = pd.merge(trainWheel, wheelbase_max, on='class_id')
trainWheel = pd.merge(trainWheel, wheelbase_min, on='class_id')
trainWheel = pd.merge(trainWheel, wheelbase_mean, on='class_id')
trainClass = pd.merge(trainClass, trainWheel, on='class_id')
trainClass

C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


,class_id,brand_id_12,brand_id_49,brand_id_68,brand_id_75,brand_id_76,brand_id_98,brand_id_106,brand_id_126,brand_id_236,...,car_height_mean,total_quality_max,total_quality_min,total_quality_mean,equipment_quality_max,equipment_quality_min,equipment_quality_mean,wheelbase_max,wheelbase_min,wheelbase_mean
0,125403,0,0,0,0,0,0,0,0,0,...,0.682875,1.000000,0.880342,0.945055,0.976852,0.847222,0.910450,0.973262,0.973262,0.973262
1,136916,0,0,0,0,0,0,1,0,0,...,0.118654,0.666667,0.555556,0.621083,0.689815,0.467593,0.551257,0.554813,0.554813,0.554813
2,178529,0,0,0,0,0,1,0,0,0,...,0.773700,0.384615,0.170940,0.269231,0.134259,0.000000,0.053241,0.521390,0.187166,0.354278
3,194450,0,0,0,0,0,0,0,0,0,...,0.463303,0.683761,0.461538,0.559829,0.650000,0.519444,0.579894,0.414439,0.360963,0.387701
4,198427,1,0,0,0,0,0,0,0,0,...,0.114679,0.230769,0.222222,0.227066,0.293519,0.254630,0.270679,0.323529,0.320856,0.322193
5,209945,0,0,0,0,0,0,0,0,0,...,0.134862,0.858974,0.730769,0.791764,0.879630,0.680556,0.789062,1.000000,1.000000,1.000000
6,248352,0,0,0,0,0,0,0,0,0,...,0.082569,0.179487,0.094017,0.136752,0.203704,0.105556,0.145885,0.320856,0.280749,0.300802
7,281301,0,0,0,0,0,0,0,0,0,...,0.115596,0.705128,0.534188,0.616667,0.675926,0.472222,0.563552,0.592246,0.592246,0.592246
8,290854,0,0,0,0,0,0,0,0,0,...,0.492202,0.649573,0.564103,0.611111,0.679630,0.509259,0.600195,0.401070,0.347594,0.374332
9,291086,0,0,0,0,0,0,0,0,0,...,0.107798,0.636752,0.470085,0.570750,0.680556,0.467593,0.600926,0.627005,0.504011,0.565508


2.25 前轮距

In [55]:
trainFTrack = train[['class_id', 'front_track']].drop_duplicates().reset_index(drop=True)
trainFTrack['front_track'] = MinMaxScaler().fit_transform(trainFTrack['front_track'])
front_track_max = trainFTrack.groupby(trainFTrack['class_id'])[['front_track']].agg('max').reset_index()
front_track_max.rename(columns={'front_track':'front_track_max'},inplace=True)
front_track_min = trainFTrack.groupby(trainFTrack['class_id'])[['front_track']].agg('min').reset_index()
front_track_min.rename(columns={'front_track':'front_track_min'},inplace=True)
front_track_mean = trainFTrack.groupby(trainFTrack['class_id'])[['front_track']].agg('mean').reset_index()
front_track_mean.rename(columns={'front_track':'front_track_mean'},inplace=True)
trainFTrack = trainFTrack.drop(['front_track'],axis=1)
trainFTrack = trainFTrack.drop_duplicates().reset_index(drop=True)
trainFTrack = pd.merge(trainFTrack, front_track_max, on='class_id')
trainFTrack = pd.merge(trainFTrack, front_track_min, on='class_id')
trainFTrack = pd.merge(trainFTrack, front_track_mean, on='class_id')
trainClass = pd.merge(trainClass, trainFTrack, on='class_id')
trainClass

C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


,class_id,brand_id_12,brand_id_49,brand_id_68,brand_id_75,brand_id_76,brand_id_98,brand_id_106,brand_id_126,brand_id_236,...,total_quality_mean,equipment_quality_max,equipment_quality_min,equipment_quality_mean,wheelbase_max,wheelbase_min,wheelbase_mean,front_track_max,front_track_min,front_track_mean
0,125403,0,0,0,0,0,0,0,0,0,...,0.945055,0.976852,0.847222,0.910450,0.973262,0.973262,0.973262,0.885333,0.816000,0.848667
1,136916,0,0,0,0,0,0,1,0,0,...,0.621083,0.689815,0.467593,0.551257,0.554813,0.554813,0.554813,0.786667,0.786667,0.786667
2,178529,0,0,0,0,0,1,0,0,0,...,0.269231,0.134259,0.000000,0.053241,0.521390,0.187166,0.354278,0.026667,0.000000,0.013333
3,194450,0,0,0,0,0,0,0,0,0,...,0.559829,0.650000,0.519444,0.579894,0.414439,0.360963,0.387701,0.906667,0.720000,0.813333
4,198427,1,0,0,0,0,0,0,0,0,...,0.227066,0.293519,0.254630,0.270679,0.323529,0.320856,0.322193,0.560000,0.538667,0.549333
5,209945,0,0,0,0,0,0,0,0,0,...,0.791764,0.879630,0.680556,0.789062,1.000000,1.000000,1.000000,0.866667,0.837333,0.855333
6,248352,0,0,0,0,0,0,0,0,0,...,0.136752,0.203704,0.105556,0.145885,0.320856,0.280749,0.300802,0.650667,0.557333,0.608000
7,281301,0,0,0,0,0,0,0,0,0,...,0.616667,0.675926,0.472222,0.563552,0.592246,0.592246,0.592246,0.792000,0.792000,0.792000
8,290854,0,0,0,0,0,0,0,0,0,...,0.611111,0.679630,0.509259,0.600195,0.401070,0.347594,0.374332,0.856000,0.760000,0.816000
9,291086,0,0,0,0,0,0,0,0,0,...,0.570750,0.680556,0.467593,0.600926,0.627005,0.504011,0.565508,0.861333,0.813333,0.837333


2.26 后轮距

In [56]:
trainRTrack = train[['class_id', 'rear_track']].drop_duplicates().reset_index(drop=True)
trainRTrack['rear_track'] = MinMaxScaler().fit_transform(trainRTrack['rear_track'])
rear_track_max = trainRTrack.groupby(trainRTrack['class_id'])[['rear_track']].agg('max').reset_index()
rear_track_max.rename(columns={'rear_track':'rear_track_max'},inplace=True)
rear_track_min = trainRTrack.groupby(trainRTrack['class_id'])[['rear_track']].agg('min').reset_index()
rear_track_min.rename(columns={'rear_track':'rear_track_min'},inplace=True)
rear_track_mean = trainRTrack.groupby(trainRTrack['class_id'])[['rear_track']].agg('mean').reset_index()
rear_track_mean.rename(columns={'rear_track':'rear_track_mean'},inplace=True)
trainRTrack = trainRTrack.drop(['rear_track'],axis=1)
trainRTrack = trainRTrack.drop_duplicates().reset_index(drop=True)
trainRTrack = pd.merge(trainRTrack, rear_track_max, on='class_id')
trainRTrack = pd.merge(trainRTrack, rear_track_min, on='class_id')
trainRTrack = pd.merge(trainRTrack, rear_track_mean, on='class_id')
trainClass = pd.merge(trainClass, trainRTrack, on='class_id')
trainClass

C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


,class_id,brand_id_12,brand_id_49,brand_id_68,brand_id_75,brand_id_76,brand_id_98,brand_id_106,brand_id_126,brand_id_236,...,equipment_quality_mean,wheelbase_max,wheelbase_min,wheelbase_mean,front_track_max,front_track_min,front_track_mean,rear_track_max,rear_track_min,rear_track_mean
0,125403,0,0,0,0,0,0,0,0,0,...,0.910450,0.973262,0.973262,0.973262,0.885333,0.816000,0.848667,0.881890,0.803150,0.829921
1,136916,0,0,0,0,0,0,1,0,0,...,0.551257,0.554813,0.554813,0.554813,0.786667,0.786667,0.786667,0.708661,0.708661,0.708661
2,178529,0,0,0,0,0,1,0,0,0,...,0.053241,0.521390,0.187166,0.354278,0.026667,0.000000,0.013333,0.000000,0.000000,0.000000
3,194450,0,0,0,0,0,0,0,0,0,...,0.579894,0.414439,0.360963,0.387701,0.906667,0.720000,0.813333,0.895013,0.682415,0.788714
4,198427,1,0,0,0,0,0,0,0,0,...,0.270679,0.323529,0.320856,0.322193,0.560000,0.538667,0.549333,0.472441,0.451444,0.461942
5,209945,0,0,0,0,0,0,0,0,0,...,0.789062,1.000000,1.000000,1.000000,0.866667,0.837333,0.855333,0.895013,0.811024,0.869423
6,248352,0,0,0,0,0,0,0,0,0,...,0.145885,0.320856,0.280749,0.300802,0.650667,0.557333,0.608000,0.627297,0.532808,0.583552
7,281301,0,0,0,0,0,0,0,0,0,...,0.563552,0.592246,0.592246,0.592246,0.792000,0.792000,0.792000,0.682415,0.682415,0.682415
8,290854,0,0,0,0,0,0,0,0,0,...,0.600195,0.401070,0.347594,0.374332,0.856000,0.760000,0.816000,0.858268,0.721785,0.797244
9,291086,0,0,0,0,0,0,0,0,0,...,0.600926,0.627005,0.504011,0.565508,0.861333,0.813333,0.837333,0.832021,0.779528,0.794619


2.27 额定载客

In [57]:
train['rated_passenger'].unique()

array(['7', '5', '7-8', '6-8', '6-7', '5-8', '5-7', '4-5', '9', '4'], dtype=object)

In [58]:
trainPassenger = train[['class_id', 'rated_passenger']].drop_duplicates()
rated_passenger_dummies = pd.get_dummies(trainPassenger['rated_passenger'], prefix='rated_passenger')
trainPassenger = pd.concat([trainPassenger,rated_passenger_dummies],axis=1)
trainPassenger = trainPassenger.drop(['rated_passenger'],axis=1)
trainPassenger = trainPassenger.groupby('class_id').agg('sum').reset_index()
trainClass = pd.merge(trainClass, trainPassenger, on='class_id')
trainClass

,class_id,brand_id_12,brand_id_49,brand_id_68,brand_id_75,brand_id_76,brand_id_98,brand_id_106,brand_id_126,brand_id_236,...,rated_passenger_4,rated_passenger_4-5,rated_passenger_5,rated_passenger_5-7,rated_passenger_5-8,rated_passenger_6-7,rated_passenger_6-8,rated_passenger_7,rated_passenger_7-8,rated_passenger_9
0,125403,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,136916,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
2,178529,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,1,0,0
3,194450,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,198427,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
5,209945,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
6,248352,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
7,281301,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
8,290854,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
9,291086,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


2.28 合并特征

In [59]:
trainClass.to_csv('train_feature.csv', index=False)